In [79]:
import pandas as pd
from scipy.io import loadmat

import h5py

In [80]:
#data_path_original = "Brown-Dwarfs/dataset_bd_wnames.csv"
mat_file = "original_dataset/ReadBrownDwarf.mat"

f = h5py.File(mat_file, 'r')
f.keys()


<KeysViewHDF5 ['#refs#', '#subsystem#', 'MCC', 'Perf', 'Precision', 'Recall', 'ReferenceResult', 'Result', 'a', 'ans', 'b', 'confMatrix', 'data', 'feat', 'fid', 'fold', 'idTE', 'idTR', 'indices', 'label', 'labelTE', 'labelTR', 'pat', 'pattern', 'rocObj', 'scoreLibSVMnoRed', 'test', 'tokens', 'train']>

In [81]:
with h5py.File(mat_file, 'r') as f:
    data = f['data'][:]
    df_data = pd.DataFrame(data).T

df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5669 entries, 0 to 5668
Data columns (total 26 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       5669 non-null   float64
 1   1       5669 non-null   float64
 2   2       5669 non-null   float64
 3   3       5669 non-null   float64
 4   4       5669 non-null   float64
 5   5       5669 non-null   float64
 6   6       5669 non-null   float64
 7   7       5669 non-null   float64
 8   8       5669 non-null   float64
 9   9       5669 non-null   float64
 10  10      5669 non-null   float64
 11  11      5669 non-null   float64
 12  12      5669 non-null   float64
 13  13      5669 non-null   float64
 14  14      5669 non-null   float64
 15  15      5669 non-null   float64
 16  16      5669 non-null   float64
 17  17      5669 non-null   float64
 18  18      5669 non-null   float64
 19  19      5669 non-null   float64
 20  20      5669 non-null   float64
 21  21      5669 non-null   float64
 22  

In [82]:
with h5py.File(mat_file, 'r') as f:
    # Access the dataset
    data = f['labelTR']

    # Access a specific reference
    ref = data[0, 0]  # Change the indices to access different references

    # Dereference the reference
    element = f[ref]
    df_label_tr = pd.DataFrame(element)
    print(element)

df_label_tr.info()

<HDF5 dataset "q": shape (4535, 1), type "<f8">
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4535 entries, 0 to 4534
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       4535 non-null   float64
dtypes: float64(1)
memory usage: 35.6 KB


In [83]:
with h5py.File(mat_file, 'r') as f:
    # Access the dataset
    data = f['labelTE']

    # Access a specific reference
    ref = data[0, 0]  # Change the indices to access different references

    # Dereference the reference
    element = f[ref]
    df_label_te = pd.DataFrame(element)
    print(element)

df_label_te.info()

<HDF5 dataset "l": shape (1134, 1), type "<f8">
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1134 entries, 0 to 1133
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       1134 non-null   float64
dtypes: float64(1)
memory usage: 9.0 KB


In [84]:
tr_unique = df_label_tr[0].unique()
tr_unique

array([2., 1.])

In [85]:
te_unique = df_label_te[0].unique()
te_unique

array([2., 1.])

In [86]:
ones_tr = df_label_tr[df_label_tr[0] == 1].count()
twoes_tr = df_label_tr[df_label_tr[0] == 2].count()
ones_tr, twoes_tr

(0    1278
 dtype: int64,
 0    3257
 dtype: int64)

In [87]:
ones_te = df_label_te[df_label_te[0] == 1].count()
twoes_te = df_label_te[df_label_te[0] == 2].count()
ones_te, twoes_te

(0    320
 dtype: int64,
 0    814
 dtype: int64)

In [88]:
df_label_tr[df_label_tr[0] == 1] = 0
df_label_tr[df_label_tr[0] == 2] = 1

df_label_te[df_label_te[0] == 1] = 0
df_label_te[df_label_te[0] == 2] = 1



In [89]:
df_data.shape, df_label_tr.shape, df_label_te.shape

((5669, 26), (4535, 1), (1134, 1))

Assume the first 4535 rows of the df_data dataframe should be mapped to df_label_tr dataframe and that the last 1134 rows should be mapped to df_label_te

In [90]:
df_label = pd.concat([df_label_tr, df_label_te])
df_label.reset_index(drop=True, inplace=True)
df_label.shape

(5669, 1)

In [91]:
df_data["label"] = df_label[0]


In [92]:
df_data

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,label
0,326.808170,2.686124,5.5560,0.0000,5.7600,0.0000,5.9470,0.0000,6.0720,0.0000,...,0.050,0.015,0.044,5.562,5.539,5.479,0.017,0.026,0.020,1.0
1,321.200440,25.880459,0.0000,0.0000,7.6116,0.0010,0.0000,0.0000,7.5167,0.0010,...,0.020,0.016,0.059,6.896,6.921,6.932,0.026,0.024,0.016,1.0
2,144.038450,-12.459262,10.5120,0.0000,10.6920,0.0000,10.8660,0.0000,10.9890,0.0000,...,0.020,0.074,0.000,10.408,10.400,10.378,0.026,0.021,0.023,1.0
3,209.662170,21.696203,0.0000,0.0000,0.0000,0.0000,6.8603,0.1042,0.0000,0.0000,...,0.046,0.015,0.037,5.671,5.717,5.704,0.023,0.063,0.020,1.0
4,65.162550,-20.639620,0.0000,0.0000,0.0000,0.0000,6.3206,0.0000,5.9504,0.0031,...,0.072,0.015,0.036,5.430,5.402,5.333,0.054,0.036,0.017,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5664,13.245956,49.443574,9.1227,0.2347,9.1823,0.0010,0.0000,0.0000,9.0066,0.0010,...,0.019,0.021,0.144,8.358,8.389,8.398,0.027,0.017,0.016,0.0
5665,29.647854,48.432832,0.0000,0.0000,9.5538,0.0010,0.0000,0.0000,10.7253,0.0010,...,0.019,0.026,0.297,8.778,8.804,8.766,0.021,0.016,0.020,1.0
5666,346.990957,54.326940,0.0000,0.0000,9.8405,0.0010,9.2025,0.0010,10.1089,0.0374,...,0.020,0.030,0.327,9.180,9.187,9.180,0.021,0.020,0.022,1.0
5667,331.173131,46.427253,10.4915,0.0630,10.2538,0.0239,10.0057,0.0074,10.4418,0.0315,...,0.020,0.033,0.000,9.693,9.678,9.622,0.024,0.029,0.023,1.0


In [93]:
df_data.to_csv('original_dataset/matlab_file_sorted.csv', index=False)

Old data visulaized.

Read the data and find the different vectors

In [94]:
# data_path = ".\original_dataset\dataset_bd_wnames.csv" #Windows
data_path = "original_dataset/dataset_bd_wnames.csv"  #MACos
df = pd.read_csv(data_path)
df.head()
df.iloc[1,:]

ID           HD 203977
sp_type             A0
RAdeg        321.20044
DEdeg        25.880459
PS1gmag            NaN
e_PS1gmag          NaN
PS1rmag         7.6116
e_PS1rmag        0.001
PS1imag            NaN
e_PS1imag          NaN
PS1zmag         7.5167
e_PS1zmag        0.001
PS1ymag        13.5874
e_PS1ymag       0.0028
W1mag             6.88
W2mag            6.919
W3mag            6.946
W4mag            6.484
e_W1mag          0.059
e_W2mag           0.02
e_W3mag          0.016
e_W4mag          0.059
Jmag             6.896
Hmag             6.921
Kmag             6.932
e_Jmag           0.026
e_Hmag           0.024
e_Kmag           0.016
label                0
Name: 1, dtype: object

Fill missing values with the mean of the vector

In [95]:
for column in df.columns:
    if column in ["ID", "sp_type"]: #Skipping these
        continue
  
    df[column] = df[column].fillna(df[column].mean()) #Imputing NA values with mean

df.head()

,ID,sp_type,RAdeg,DEdeg,PS1gmag,e_PS1gmag,PS1rmag,e_PS1rmag,PS1imag,e_PS1imag,...,e_W2mag,e_W3mag,e_W4mag,Jmag,Hmag,Kmag,e_Jmag,e_Hmag,e_Kmag,label
0,* 11 Peg,A0V,326.80817,2.686124,5.55600,0.02303,5.760000,0.027126,5.947000,0.022877,...,0.050,0.015,0.044000,5.562,5.539,5.479,0.017,0.026,0.020,0
1,HD 203977,A0,321.20044,25.880459,15.12594,0.02303,7.611600,0.001000,14.918063,0.022877,...,0.020,0.016,0.059000,6.896,6.921,6.932,0.026,0.024,0.016,0
2,HD 83162,A0III/IV,144.03845,-12.459262,10.51200,0.02303,10.692000,0.027126,10.866000,0.022877,...,0.020,0.074,0.099197,10.408,10.400,10.378,0.026,0.021,0.023,0
3,* 10 Boo,A0Vs,209.66217,21.696203,15.12594,0.02303,14.982256,0.027126,6.860300,0.104200,...,0.046,0.015,0.037000,5.671,5.717,5.704,0.023,0.063,0.020,0
4,HD 27616,A0V,65.16255,-20.639620,15.12594,0.02303,14.982256,0.027126,6.320600,0.000000,...,0.072,0.015,0.036000,5.430,5.402,5.333,0.054,0.036,0.017,0


Encode the vector "sp_type" as it is categorical

In [96]:
X = df.drop('ID', axis=1)
unique_values = len(X["sp_type"].unique()) # 504
X['sp_type'], _ = pd.factorize(X['sp_type'])

Split the dataframe/array into data and ground truth

In [97]:
y = X["label"]
X = X.drop(columns=["label"])

Using the actual dataset from the matlab file given instead. Overwriting the previous values of data_path, X, y. 

In [98]:
data_path = "original_dataset/matlab_file_sorted.csv"  #MACos
# data_path = ".\original_dataset\matlab_file_sorted.csv.csv" #Windows
df = pd.read_csv(data_path)
y = df["label"]
X = df.drop(columns=["label"])

In [99]:
X.shape,y.shape

((5669, 26), (5669,))